# Transformers - from top to the bottom

En este post vamos a ver cómo funcionan los Transformers de arriba a abajo. Pero si te interesa aprendelo de abajo a arriba lo puedes ver en este post [Transformers - from bottom to the top](https://www.maximofn.com/transformers-from-bottom-to-the-top).

## Transformer como una caja negra

La arquitectura transformer se creo para el problema de traducción, por lo que vamos a explicarlo para ese problema

Imaginemos el transformer como una caja negra, a la que le entra una frase en un idioma y saca la misma frase traducida en otro idioma.

![Transformer - black box](http://maximofn.com/wp-content/uploads/2024/02/Transformer-black-box.png)

## Tokenización

Pero como hemos visto en el post de [tokens](https://maximofn.com/tokens/), los modelos de lenguaje no entienden las palabras como nosotros, sino que necesitan números para poder realizar las operaciones. Por lo que la frase en el idioma original se tiene que convertir a tokens mediante un tokenizador, y a la salida necesitamos un detokenizador para convertir los tokens de salida a palabras

![Transformer - black box - tokenizers](http://maximofn.com/wp-content/uploads/2024/02/Transformer-black-box-tokenizers.png)

De modo que el tokenizador crea una secuencia de $n_{input-tokens}$ tokens, y el detokenizador recibe una secuencia de $n_{output-tokens}$ tokens.

## Input embeddings

En el post de [embeddings](https://maximofn.com/embeddings/) vimos que los embeddings son una forma de representar las palabras en un espacio vectorial. Por lo que los tokens de entrada se pasan por una capa de embeddings para convertirlos en vectores.

En un resumen rápido, el proceso de embedding consiste en convertir una secuencia de números (tokens) en una secuencia de vectores de. Si teníamos $n_{input-tokens}$ tokens, ahora tenemos $n_{input-tokens}$ vectores. Cada uno de esos vectores tiene una longitud de $d_{model}$. Es decir, se cada token se convierte a un vector que representa ese token en un espacio vectorial de $d_{model}$ dimensiones.
Por tanto después de pasar por la capa de embeddings, la secuencia de $n_{input-tokens}$ tokens se convierte en una matriz de ($n_{input-tokens}$ x $d_{model}$).

![Transformer - black box - input embeddings](http://maximofn.com/wp-content/uploads/2024/02/Transformer-black-box-input-embeddings.png)

## Encoder - decoder

Hemos visto el transformer actuando como una caja negra, pero en realidad el transformer es una arquitectura que se compone de dos partes, un encoder y un decoder.

![Transformer - encoder-decoder](http://maximofn.com/wp-content/uploads/2024/02/Transformer-encoder-decoder.png)

El encoder se encarga de comprimir la información de la frase de entrada, crea un espacio latente donde está esa información de la frase de entrada comprimida. A continuación, esa información comprimida entra al decoder, que sabe convertir esa información comprimida en una frase del idioma de salida.

Y ¿cómo convierte el decoder esa información comprimidaen una frase del idioma de salida? Pues token a token. Para entenderlo mejor vamos a olvidarnos de los tokens de salida por un momento, vamos a imaginar que tenemos esta arquitectura

![Transformer - encoder-decoder (no detokenizer)](http://maximofn.com/wp-content/uploads/2024/02/Transformer-encoder-decoder-no-detokenizer.png)

Es decir, la frase del idioma original se convierte a tokens, estos tokens se convierten a embeddings, que entran al encoder, este comprime la información, el decoder la coge y la convierte en palabras del idioma de salida

De modo que el decoder va generando una palabra nueva a la salida en cada paso

![Transformer - encoder-decoder (no detokenizer)](https://raw.githubusercontent.com/maximofn/portafolio/main/images/Transformer%20-%20encoder-decoder%20(no%20detokenizer).gif)

Pero ¿cómo sabe el decoder cúal es la palabra que tiene que generar cada vez? Porque se le está pasando la frase que ya ha traducido, y en cada paso va generando la siguiente palabra. Es decir, en cada paso el decoder recibe la frase que ha traducido hasta el momento, y genera la siguiente palabra.

Pero aun así, ¿cómo sabe que tiene que generar la primera palabra? Porque se le pasa una palabra especial que significa "empezar a traducir", y a partir de ahí va generando las siguientes palabras.

Y por último, ¿cómo sabe el transformer que tiene que dejar de generar palabras? Porque cuando termina de traducir genera una palabra especial que significa "fin de la traducción", que cuando vuelve a entrar al transformer hace que no genere más palabras.

![Transformer - encoder-decoder (no detokenizer) (input)](https://raw.githubusercontent.com/maximofn/portafolio/main/images/Transformer%20-%20encoder-decoder%20(no%20detokenizer)%20(input).gif)

Ahora que lo hemos entendido con palabras, que es más sencillo, vamos a volver a colocar el detokenizador a la salida

![Transformer - encoder-decoder](http://maximofn.com/wp-content/uploads/2024/02/Transformer-encoder-decoder.png)

Por tanto el decoder irá generando tokens. Para saber que tiene que empezar una frase se le mete un token especial comunmente llamado `SOS` (Start Of Sentence), y para saber que tiene que terminar genera otro token especial comunmente llamado `EOS` (End Of Sentence).

Y al igual que el encoder, el token de entrada tiene que pasar por una capa de embedding para convertir los tokens en representaciones vectoriales.

Suponiendo que cada token equivale a una palabra, el proceso de traducción sería el siguiente

![Transformer - encoder-decoder (detokenizer)](https://raw.githubusercontent.com/maximofn/portafolio/main/images/Transformer%20-%20encoder-decoder%20(detokenizer).gif)

De momento tenemos esta arquitectura

![Transformer - encoder-decoder (detokenizer)](http://maximofn.com/wp-content/uploads/2024/02/Transformer-encoder-decoder-detokenizer-2.png)

## Projection

Hemos dicho que al decoder le entra un token que pasa por la capa de embedding `Output embedding`.

El `Output decoder` crea un vector por cada token, por lo que a la salida del `Output decoder` tenemos una matriz de ($n_{output-tokens}$ x $d_{model}$).

El decoder hace operaciones, pero saca una matriz con la misma dimensión. Así de necesita convertir esa matriz en un token y eso lo hace mediante una capa lineal que a la salida genera un array con la misma dimensión que los posibles tokens que hay en el lenguaje al que se quiere traducir (vocabulario de salida).

Ese array corresponde a los logits de cada posible token, por lo que a continuación se pasa por una capa softmax que convierte esos logits en probabilidades. Es decir, tendremos la probabilidad de que cada token sea el siguiente token.

![Transformer - projection](http://maximofn.com/wp-content/uploads/2024/02/Transformer-projection.png)

## Encoder y decoder x6

En el paper original usan 6 capas para el encoder y otras 6 capas para el decoder. No hay ninguna razón para que sean 6, supongo que probaron varios valores y este fue el que mejor les funcionó.

A cada uno de los decoder le entra la salida del último encoder

![Transformer - encoder-decoder (x6)](http://maximofn.com/wp-content/uploads/2024/02/Transformer-encoder-decoder-x6.png)

Para simplificar el diagrama lo representaremos así a partir de ahora

![Transformer - encoder-decoder (Nx)](http://maximofn.com/wp-content/uploads/2024/02/Transformer-encoder-decoder-Nx.png)

## Attention - Feed forward

Vamos a ir empezando a ver qué ha dentro del encoder y el decoder. Básicamente lo que hay es un mecanismo de atención y una capa feed forward.

![Transformer - encoder-decoder - attention-ff](http://maximofn.com/wp-content/uploads/2024/02/Transformer-encoder-decoder-attention-ff.png)

### Atención

Podemos ver que en los mecanismos de atención entran 3 flechas. Esto ya lo veremos más adelante cuando veamos en profundidad cómo funcionan los mecanismos de atención.

Pero de momento podemos decir que son operaciones que se realizan para poder obtener la relación que existe entre los tokens (y por tanto, la relación que existe entre las palabras).

Antes de los transformers, para el problema de traducción se usaban las redes neuronales recurrentes, que consistían en redes a las que les entraba un token, lo procesaban y generaban otro token de salida. A continuación le entraba un segundo token, lo procesaban y sacaban otro otken, y así sucesivamente con todos los tokens de la secuencia de entrada. El problema de estas redes es que cuando las frases eran muy largas, cuando se estaba en los últimos tokens, la red se "olvidaba" de los primeros. Por ejemplo en frases muy largas, podría pasar que se cambiase el género del sujeto a lo largo de la frase traducida. Y esto es porque después de muchos tokens, la red se había olvidado si el sujeto era masculino o femenino.

Para solucionar esto, en el mecanismo de atención de los transformers entra la secuencia entera y de una sola vez se calculan las relaciones (atención) entre todos los tokens. 

Esto es muy potente, ya que en un solo cálculo se obtiene la relación entre todos los tokens, sea lo larga que sea la secuencia.

Por ejemplo en esta imagen, se puede ver como la palabra `it` tiene relación con palabras distintas en función de la frase entera

![Transformer - attention sentences](http://maximofn.com/wp-content/uploads/2024/02/Transformer-attention-sentences.png)

Aunque esto es una gran ventaja y es lo que ha hecho que los transformers ahora se utilicen en la mayoría de las mejores redes modernas, también es su mayor desventaja, ya que el cálculo de la atención es muy costoso computacionalmente. Ya que requiere unas multiplicaciones matriciales muy grandes.

Esas multiplicaciones se realizan entre matrices que corresponden a los embeddings de cada uno de los tokens por ellas mismas. Es decir, la matriz que representa los embeddings de los tokens se multiplica por ella misma. Para poder realizar esta multiplicación hay que rotar una de las matrices (requisitos del algebra para poder multiplicar matrices).

Por lo que a medida que aumenta la longitud de las secuencias, la cantidad de memoria necesaria para almacenar esas matrices crece de forma cuadrática. Y esto es un gran limitante a día de hoy, la cantidad de memoria que tienen las GPUs, que es dónde se suelen realizar esas multiplicaciones.

En el encoder se usa una sola capa de atención, para sacar las relaciones entre los tokens de entrada

En el decoder se utilizan dos capas de atención, una para sacar las relaciones entre los tokens de salida, y otra para sacar las relaciones entre los tokens del encoder y los del decoder.

### Feed forward

Después de la capa de atención, esta capa la veremos más a fondo más adelante, pero de momento podemos decir que tiene dos propósitos

 * Uno es añadir no linealidades. Como hemos explicado, la atención se consigue mediante multiplicaciones matriciales de los tokens de las secuencias de entrada. Pero si a una red no se le aplican capas no lineales, al final, toda la arquitectura se podría resumir en unos pocos cálculos lineales. Por lo que las redes neuronales no podrían resolveer problemas no lineales. De modo que se añade esta capa para añadir no linealidad

  * Otro es extracción de características. Aunque la atención ya extrae características, estas son características de las relaciones entre los tokens. Pero esta capa se encarga de extraer características de los tokens en sí. Es decir, de cada token se extraen características que se consideran importantes para el problema que se está resolviendo.